In [25]:
import numpy as np

# quara
from quara.objects.matrix_basis import (
    get_normalized_pauli_basis,
)
from quara.objects.elemental_system import ElementalSystem
from quara.objects.composite_system import CompositeSystem
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)

In [2]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
print(c_sys)

elemental_systems:
[0] 0 (system_id=4434319440)

dim: 2
basis:
(array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]]))


In [3]:
# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)
for i, tester in enumerate(testers):
    print("Tester ", i, ":\n", tester)

Tester  0 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]]
Tester  1 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]]
Tester  2 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [4]:
# Quantum State Tomography
seed = 7896
qst = StandardQst(testers, on_para_eq_constraint=True, schedules="all", seed=seed)

In [10]:
mode = "state"
name = "a"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
State

Dim:
2

Vec:
[0.70710678 0.5        0.5        0.        ]


In [11]:
prob_dists = qst.calc_prob_dists(true)
print(prob_dists)

[[0.85355339 0.14644661]
 [0.85355339 0.14644661]
 [0.5        0.5       ]]


In [23]:
num_data = 1000
empi_dists = qst.generate_empi_dists(state=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.881, 0.119]))
(1000, array([0.859, 0.141]))
(1000, array([0.493, 0.507]))


In [24]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.53881537  0.50770267 -0.00989949]
is estimate physical? :  False


In [ ]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(3)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
    )
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())

In [5]:
# POVM Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names = ["x0", "y0", "z0", "z1"]
testers = generate_tester_states(c_sys=c_sys, names = names)
qt = StandardPovmt(testers, num_outcomes=4, on_para_eq_constraint=True, schedules="all")

for i in range(qt.num_schedules):
    print("i=", i, ", num_outcomes = ", qt.num_outcomes(i))

i= 0 , num_outcomes =  4
i= 1 , num_outcomes =  4
i= 2 , num_outcomes =  4
i= 3 , num_outcomes =  4
i= 4 , num_outcomes =  4
i= 5 , num_outcomes =  4
i= 6 , num_outcomes =  4
i= 7 , num_outcomes =  4
i= 8 , num_outcomes =  4
i= 9 , num_outcomes =  4
i= 10 , num_outcomes =  4
i= 11 , num_outcomes =  4
i= 12 , num_outcomes =  4
i= 13 , num_outcomes =  4
i= 14 , num_outcomes =  4
i= 15 , num_outcomes =  4


In [6]:
# Quantum Process Tomography

# Composite System
mode = "qubit"
num = 2
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names_states = ["x0", "y0", "z0", "z1"]
testers_states = generate_tester_states(c_sys=c_sys, names = names_states)
names_povms = ["x", "y", "z"]
testers_povms = generate_tester_povms(c_sys=c_sys, names=names_povms)
qt = StandardQpt(states=testers_states, povms=testers_povms, on_para_eq_constraint=True, schedules="all")

for i in range(qt.num_schedules):
    print("i=", i, ", num_outcomes = ", qt.num_outcomes(i))

i= 0 , num_outcomes =  4
i= 1 , num_outcomes =  4
i= 2 , num_outcomes =  4
i= 3 , num_outcomes =  4
i= 4 , num_outcomes =  4
i= 5 , num_outcomes =  4
i= 6 , num_outcomes =  4
i= 7 , num_outcomes =  4
i= 8 , num_outcomes =  4
i= 9 , num_outcomes =  4
i= 10 , num_outcomes =  4
i= 11 , num_outcomes =  4
i= 12 , num_outcomes =  4
i= 13 , num_outcomes =  4
i= 14 , num_outcomes =  4
i= 15 , num_outcomes =  4
i= 16 , num_outcomes =  4
i= 17 , num_outcomes =  4
i= 18 , num_outcomes =  4
i= 19 , num_outcomes =  4
i= 20 , num_outcomes =  4
i= 21 , num_outcomes =  4
i= 22 , num_outcomes =  4
i= 23 , num_outcomes =  4
i= 24 , num_outcomes =  4
i= 25 , num_outcomes =  4
i= 26 , num_outcomes =  4
i= 27 , num_outcomes =  4
i= 28 , num_outcomes =  4
i= 29 , num_outcomes =  4
i= 30 , num_outcomes =  4
i= 31 , num_outcomes =  4
i= 32 , num_outcomes =  4
i= 33 , num_outcomes =  4
i= 34 , num_outcomes =  4
i= 35 , num_outcomes =  4
i= 36 , num_outcomes =  4
i= 37 , num_outcomes =  4
i= 38 , num_outcomes =